In [1]:
import numpy as np
from keras.models import Model
from keras.layers import Input, Flatten, Conv2D, MaxPooling2D, BatchNormalization, Dense, ZeroPadding2D, Activation
import cv2
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
import warnings
from os import *
warnings.filterwarnings('ignore')

In [2]:
x = []
y = []
for directory in ['augmented data/yes', 'augmented data/no']:
	for filename in listdir(directory):
		image = cv2.imread(directory + '/' + filename)
		image = cv2.resize(image, dsize=(240,240), interpolation=cv2.INTER_CUBIC)
		image = image / 255.
		x.append(image)
		if directory[-3:] == 'yes':
			y.append([1])
		else:
			y.append([0])

In [3]:
x = np.array(x)
y = np.array(y)
x, y = shuffle(x, y)

print(f'Number of samples : {len(x)}')
print(f'x shape : {x.shape}')
print(f'y shape : {y.shape}')

Number of samples : 2065
x shape : (2065, 240, 240, 3)
y shape : (2065, 1)


In [4]:
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.2, random_state=101)

In [5]:
input = Input((240,240,3))
model = ZeroPadding2D((2, 2))(input)
model = Conv2D(32, (7, 7), strides = (1, 1), name = 'conv0')(model)
model = BatchNormalization(axis = 3, name = 'bn0')(model)
model = Activation('relu')(model)
model = MaxPooling2D((4, 4), name='max_pool0')(model) 
model = MaxPooling2D((4, 4), name='max_pool1')(model)
model = Flatten()(model)
model = Dense(1, activation='sigmoid', name='fc')(model)
model = Model(inputs = input, outputs = model, name='BrainTumorDetection')
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [6]:
print(model.summary())

Model: "BrainTumorDetection"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 240, 240, 3)]     0         
                                                                 
 zero_padding2d (ZeroPadding  (None, 244, 244, 3)      0         
 2D)                                                             
                                                                 
 conv0 (Conv2D)              (None, 238, 238, 32)      4736      
                                                                 
 bn0 (BatchNormalization)    (None, 238, 238, 32)      128       
                                                                 
 activation (Activation)     (None, 238, 238, 32)      0         
                                                                 
 max_pool0 (MaxPooling2D)    (None, 59, 59, 32)        0         
                                               

In [7]:
model.fit(x=xtrain, y=ytrain, batch_size=32, epochs=10)

Epoch 1/10
52/52 [==============================] - 102s 2s/step - loss: 0.7477 - accuracy: 0.6810
Epoch 2/10
52/52 [==============================] - 97s 2s/step - loss: 0.5433 - accuracy: 0.7403
Epoch 3/10
52/52 [==============================] - 94s 2s/step - loss: 0.5259 - accuracy: 0.7439
Epoch 4/10
52/52 [==============================] - 99s 2s/step - loss: 0.4443 - accuracy: 0.7900
Epoch 5/10
52/52 [==============================] - 95s 2s/step - loss: 0.4139 - accuracy: 0.8045
Epoch 6/10
52/52 [==============================] - 95s 2s/step - loss: 0.3952 - accuracy: 0.8305
Epoch 7/10
52/52 [==============================] - 96s 2s/step - loss: 0.4223 - accuracy: 0.8045
Epoch 8/10
52/52 [==============================] - 87s 2s/step - loss: 0.4001 - accuracy: 0.8202
Epoch 9/10
52/52 [==============================] - 84s 2s/step - loss: 0.3274 - accuracy: 0.8565
Epoch 10/10
52/52 [==============================] - 94s 2s/step - loss: 0.3161 - accuracy: 0.8571


In [8]:
test_pred = np.round(model.predict(xtest))
train_pred = np.round(model.predict(xtrain))

In [9]:
def thres(pred):
	output = []
	for i in pred:
		if i > 0.8:
			output.append([1])
		else:
			output.append([0])
	return output

test_output = thres(test_pred)
train_output = thres(train_pred)

In [10]:
print('Training Data Metrics')
print(confusion_matrix(ytrain, train_output))
print(accuracy_score(ytrain, train_output))
print('Testing Data Metrics')
print(confusion_matrix(ytest, test_output))
print(accuracy_score(ytest, test_output))

Training Data Metrics
[[727  58]
 [ 99 768]]
0.9049636803874092
Testing Data Metrics
[[165  30]
 [ 47 171]]
0.8135593220338984


In [11]:
model.save('BrainTumorModel.h5')